# **Retrieval Experiment 2: Re-ranking Strategies for Retriever**

To ensure that the LLM recieves the most relevant documents from the retriever, we explore methods to re-rank the results

**Retrieval Bi-Encoder + Re-Ranker Cross-Encoder**

Bi-Encoders produce a sentence embedding for a single sentence. This is what we have been using for our embedding function of the vector store where embeddings are individually computed for documents and query, and then compared using a similarity measure.

In Cross-Encoders, we pass a query and document simultaneously to a Transformer network. It produces an output value between 0 and 1 indicating the similarity of the input sentence pair.

Bi-Encoders are much faster but less accurate than Cross-Encoders, which is why we use Bi-Encoders for efficient vector similarity calculations and Cross-Encoders to re-rank the top-k results for every query-hit pair.

LLMs face a context-stuffing/lost-in-the-middle problem. To mitigate this problem, we maximize the retriever's recall by retrieving plenty of documents and then maximize the LLM's recall by minimizing the number of documents that make it to the LLM, such that only the most relevant documents are considered. This will boost the LLM's ability to provide better results.

To do this, we take a 2-stage retrieval approach. We use Bi-Encoders for the retriever and Cross-Encoders to re-rank the retrieved documents.


In [1]:
%pip install --quiet --upgrade bitsandbytes langchain langchain-community langchain-huggingface transformers beautifulsoup4 faiss-gpu rank_bm25 lark qdrant-client langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 105.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.1 MB/s eta 0

In [2]:
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
import os
from google.colab import files
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
import pandas as pd

## **User Action Required**

1. Run the code below to create the ```data``` folder

2. Choose to upload the following files
- ```iceland_articles_updated.csv```
- ```finland_articles_updated.csv```


In [3]:
data_folder = os.path.join(os.getcwd(), 'data')
os.makedirs(data_folder, exist_ok=True)

In [4]:
uploaded_files = files.upload()

Saving iceland_articles_updated.csv to iceland_articles_updated.csv
Saving finland_articles_updated.csv to finland_articles_updated.csv


In [5]:
for file_name in uploaded_files.keys():
    os.rename(file_name, os.path.join(data_folder, file_name))

Your folder structure should now look as such:

```
data
  - iceland_articles_updated.csv
  - finland_articles_updated.csv
```

**Setup Simple Experiment Data for Retrieval Experiment 2**

We first set up simple experiment data where we have 3 documents talking about different 3 different topics. This is done to have an easier <u>preliminary understanding and visualisation</u> of the behaviour of the re-ranker as we are able to quickly look at what are the chunks being retrieved from the data and how they are ranked

In [6]:
# Simple experiment example data
docs = [
    Document(
        page_content='''
        10 new unique places to stay in Finland From modern wooden cabins and brand-new glass huts to resorts with a touch of luxury: Finland offers a plethora of unique accommodation options that showcase the country's breathtaking natural beauty both in the materials and the locations. In recent years, there has been upsurge in unique new hotels, resorts, and cabins opening up around the country. We've curated a list of the coolest and most recently opened ones where you can immerse yourself in Finnish nature and find your inner happiness. Article published in June 2023.; 1. Hilltop Forest – Inkoo, Helsinki region: Hilltop Forest, a small resort in Inkoo, offers charming triangular wooden huts with large windows facing a lush forest. Curl up in the fresh linen bedding and admire the view over the majestic pine trees and birches. The sole mission of the day is to stroll through the forest to the nearby spa. The resort's restaurant serves breakfast, lunch, freshly baked pizzas, wine, and forest-inspired dinners. Hilltop Forest is a perfect minivacation destination, just a under an hour-drive from Helsinki. Eat, sleep and repeat.; 2. The Torby – Fiskars village, Helsinki region: The brand new Torby Hotel, opened in June 2023, is nestled in the charming Fiskars Village. The Torby offers an enchanting retreat for travellers seeking tranquillity and inspiration. Set amidst the lush countryside of the Helsinki region, this boutique hotel seamlessly blends modern design with the rich cultural heritage of the Fiskars village. The village is a treasure trove of creativity, with its vibrant art studios, quaint shops, and idyllic gardens. Guests can explore the local craftsmanship, indulge in delicious Finnish cuisine, or simply immerse themselves in the serene ambiance. Whether seeking a peaceful retreat or a cultural adventure, the Torby Hotel and Fiskars Village promise an unforgettable experience.; 3. Hotel Vihannonkulma – Loimaa, Coast and Archipelago: The Hotel Vihannonkulma building, a private home of the car dealer Olli Vihanto, was closed for almost 30 years until the apartment was rediscovered and revived into a boutique hotel opened in 2022. Spend the night in a classic room with sleek Alvar Aalto inspired lines, or in a modern 70s-style room with eccentric details and teak furniture. Enjoy the small-town atmosphere of Loimaa and stroll around in cafés, shops, and historical buildings. Loimaa, located 50 minutes northeast of Turku, serves as a gateway to the surrounding forests and nature.; 4. Kanava Resort – Oravi, Lakeland: Located on a narrow headland surrounded by Lake Saimaa, Kanava Resort is a haven for happiness seekers and outdoor adventurers. The resort is a harmonious blend of contemporary design and natural beauty. The buildings, constructed with sustainable materials, seamlessly blend into the landscape, providing an immersive experience for guests. The resort offers a range of activities across the year. In the winter, take a ride on a snowmobile or slide down the terrain on a pair of skis. During summer hop on a canoe, grab the oars and slide through the waters. Make sure not to miss the most popular Lakeland adventure: spotting ringed seals together with a guide. Kanava Resort is just a short drive away from the national parks Linnansaari and Kolovesi. While you are in the area, remember to also visit the historical landmark Olavinlinna Castle and the iconic Punkaharju Ridge.; 5. PihlasResort – Joroinen, Lakeland: Located on the shore of the biggest lake in Finland, Lake Saimaa, PihlasResort is the perfect place to embrace luxury. The brand new suites, opening in summer 2023, are equipped with a kitchen, a private sauna, and a terrace facing a private garden. Relax by the lake, have a drink in a jacuzzi, or feel the rejuvenating power of an authentic Finnish sauna – relaxation is guaranteed. During your visit, dine in the resort's own restaurant Siimes, or enjoy a unique tea menu in the nearby TeaHouse of Wehmais.; 6. Villipeura’s Niliaitta – Kivijärvi, Lakeland: At Villipeura's holiday village in Kivijärvi, located less than two hours north of Jyväskylä in central Lakeland, unique birdhouse-like boxes are elevated on pillars in the midst of a charming Finnish forest. These uncommon holiday cottages draw inspiration from traditional food storage sheds that were built on wooden pillars to protect food from animals in the past. In the holiday village area, you have access to a range of activities, including tennis, frisbee golf, and fishing. Nature trails of the national parks Salamajärvi and Pyhä Häkki are just around the corner.; 7. Glass huts in Skyfire Village – Rovaniemi, Lapland: At Skyfire Village in Rovaniemi you may need to pinch yourself to figure out whether the view is real: the panoramic windows in the glass huts allow you to gaze at northern lights in autumn and winter, or the midnight sun in the summer directly from your bed. Skyfire Village is in a quiet location with no light pollution making it even easier to see the auroras. The village's own restaurant, Sky Hut Restaurant & Bar, offers tailor-made menus by a professional chef using local ingredients.; 8. Kurula’s – Pyhätunturi, Lapland: Kurula’s in Lapland's pristine Pyhä region is the perfect vacation getaway for those who want a combination of quiet relaxation and outdoors activities. In the summer, go on a hike in Finland’s oldest national park Pyhä-Luosto or ascend the fell with a skiing lift to admire the views. In the winter, embark on a husky or reindeer safari and hunt the northern lights. Your own private lakeside sauna in the suite is a perfect way to end a day full of adventures. Kurula is close to the popular outdoor and skiing resort of Pyhä.; 9. Arctra – Rovaniemi, Lapland: At Arctra's private luxury resort near Rovaniemi in Lapland, you can enjoy a glamorous vacation atop the mighty hill of Ollerovaara, with stunning views of arctic nature. Explore the landscape by foot, on skis, or on a husky sled. A private 20 hectare wilderness area will guarantee that there most likely won’t be another soul in sight. Cosy up in your stylish suite or at the spa area. Arctra is only a 15-minute drive from Rovaniemi airport. The cottage serves as an excellent home base to explore Rovaniemi as well as the surrounding wilderness.; 10. Cahkal Hotel – Kilpisjärvi, Lapland: Cahkal Hotel is located 400 kilometres above the Arctic Circle in the northwesternmost point of Finland, in the truly unique Nordic wilderness. Cahkal is an ideal destination for experiential travellers looking for a view – all the hotel rooms have exquisite mountain views. Embark on an expedition with a local guide to explore the arctic landscape and even pop by in Norway to see the fjords and the Arctic Ocean. Unwind in the hotel's sauna after an adventure of a lifetime. Cahkal's heating system operates by sustainable geothermal heating and solar panels provide visitors with green electricity.
        '''
    ),
    Document(
        page_content='''
        Finnish food culture Finland’s cuisine is built around fresh, natural ingredients gathered straight from the waters, fields and forests: Get to know the local ingredients and delicious dishes you simply can’t miss when visiting Finland. Finns are known as leaders of research and innovation. But when it comes to food, they tend to favour tradition. Finnish cuisine differs from neighbouring Scandinavian countries because it has one foot on either side of its borders – Russia in the East and Sweden in the West. Since day one, the Finnish diet has been built around surviving the harsh conditions of our northern climate. Growth seasons are dictated by the cold, meaning many local products are only available for a limited amount of time. These days, you can buy just about anything your heart desires in Finland. But go local while you’re here, and you’ll be in for a treat! Also, if you have the chance, continue your culinary journey to some of the best restaurants in the country.; Cereals and grains – made from scratch: All hail the golden crop of Finnish soil, oats! And what’s not to love about the latest boom of oat milks, creams and yoghurts, not to mention pulled oats, which are an extremely tasty alternative to meat? Make sure to taste this decidedly Finnish innovation! If you’re more into traditional foods, stick to good ol’ porridge. This hearty delight can be enjoyed at any gas station, hotel or café for just a few euros. Warming, filling and incredibly tasty, it’s best served with a spoonful of jam or a little fresh butter. Tip: When it comes to local grains, think bakery and check out pastry chef Teemu Aura. The bakeries carrying his name and signature pink bun tracks are called Pullabiili, and they can be found throughout the Helsinki region offering classic Finnish buns, flaky croissants and sourdough breads. All these treats are made from scratch and combine culinary creativity with high-quality ingredients.; Fish – another staple of the Finnish diet: Whether it’s tasty salmon soup, fillets of perch, pickled Baltic herring or smoked vendace, the list simply goes on and on. Best consumed fresh at food markets, delis and restaurants throughout the country, fish dishes are the heart of the Finnish diet. Unsure what to try? Go for smoked – it’s the archipelago’s signature way to enjoy the fruits of the sea. Or make some memories on a fishing trip, where you can catch and prepare your own. Contact the local tourist information points for tours. Tip: Crayfish parties, or “rapujuhlat” (“kräftskiva” in Swedish), are a Swedish tradition the Finns adopt during late summer’s crayfish season. These small fresh-water lobsters are considered a gourmet treat, which is why they are feted in style – often accompanied by plenty of schnapps and special crayfish songs. If you’re invited to one of these parties, do say yes!; Fresh, juicy berries are the gold of Finland’s forests: Plentiful and sweet when in season, lingonberries, woodland strawberries and blueberries taste best when they’re picked straight from the forest or purchased fresh by the litre at a local market. Off-season and outside of the warmer months of the year, local jams with cloudberries and the notorious sea-buckthorn are the preferred way to enjoy Finland’s berry bounty. Tip: Enjoy your berries dried and ground. METTÄ Nordic offers the exciting flavours of the Finnish forests in powder form. Buy a bag or two and try this delicious addition on breakfasts and snacks at home!; Mushrooms – ceps, chanterelles, false morel and list goes on..: Under the Everyman’s right, you can pick almost anything your heart desires while visiting Finland’s forests. Just be mindful and check the local rules if roaming in national parks or other protected areas. Tip: Please don’t pick mushrooms unless you’re an experienced forager. Guided mushroom-picking tours are always the safest bet for beginners.; Finnish superfood innovations have exploded onto the world’s markets: From Beanit’s plant-based protein products made from Nordic Fava beans to locally brewed kombucha drinks by Good Guys Kombucha in Pirkkala, Finland’s food companies are always thinking ahead. Tip: Want to try Finland’s superfoods? Go wild with an herb foraging class. You’ll not only return with a bag full of fresh greens, you’ll also get to try forest bathing, another Finnish. Foraging trips can be organised through companies such as Feel the Nature, and Finland, Naturally Experiences.   Hungry for more? Here’s a list of iconic foods and regional delicacies beloved by locals and visitors alike. Make sure to try them all!; Leipäjuusto – delicious dairy product: Leipäjuusto, known as “Squeaky Cheese,” is a mild, incredibly tasty cheese that’s typically made of cow’s milk. First, the milk is curdled, then it’s baked it in the oven, and finally, it’s cut into thin wedges and served. The exterior of the cheese gets its spotted black and white colouring from the heat of the oven, and it’s Finnish name means “cheese bread” (since it’s baked like bread). The end product is a yummy cheese with a deliciously squeaky consistency. Tip: Leipäjuusto is often served with cloudberry jam as dessert, but the traditional Sámi way of eating it is to dip the wedges in hot, black coffee. Try it to experience the perfect blend of smooth, fatty cheese cut by hot, bitter coffee.; Mild but full of flavour, Finnish salmon soup: Finnish salmon soup is a classic that’s served both at home and in restaurants. The most popular version features creamy white broth studded with salmon, onions and potatoes and garnished with a handful of dill. You’ll find this delicacy on the menu at many restaurants and cafés – it’s a true comfort food that will warm you up on a winter’s day.; Endless varieties of new potatoes: New potatoes with herring (silli). New potatoes with fresh lake fish and chantarelle sauce. New potatoes with fish roe (mäti). New potatoes with just a knob of butter, some dill and a little salt. The variations of new potatoes available in Finland are seemingly endless and certainly mouth-watering. You’ll find that Finns can talk about new potatoes forever, as the little spuds hold the promise of summer. Look for them starting around midsummer and expect to see statistics on the harvest in all the local papers.; Poronkäristys from Lapland: Reindeer are found in the northern province of Lapland, and according to recent research, their meat is one of the healthiest you can put on your plate. It’s high in B-12, omega-3 and omega-6 – and it’s lean and delicious! Served alongside mashed potatoes, sautéed reindeer is a Finnish treat that’s eaten throughout the country, all year round.; Kalakukko – a local delicacy: Describing this local delicacy is difficult, but it’s essentially a combination of salty vendace and fatty pork that’s then wrapped in a rye crust and baked in foil. Best served alone or with heaps of butter, this delicacy is most delicious at its origin: a busy marketplace in Kuopio in Finland’s Lakeland. Loosely translated as “fish rooster,” this iconic pie’s name makes absolutely no sense, but it tastes incredibly good! And fun fact: the name of this unique delicacy has been granted protection under the EU quality scheme, along with Karjalanpiirakka, or Karelian pie.; Karjalanpiirakka – a sublime pastry: And speaking of Karjalanpiirakka, Karelian pie is the crown jewel of Finnish cuisine. This sublime pastry is originally from the eastern province of Karelia, and with its filling of delicious rice, potatoes or carrots enclosed in an incredibly crisp rye crust, this buttery delicacy will win you over in no time. It’s best served with plenty of fresh butter or a serving of egg butter-spread. Tip: You’ll find Karelian pies at any self-respecting Finnish marketplace, café, supermarket or gas station. Fresh, unbaked pies can also be purchased and baked at home!; Ruisleipä – made from sour dough: Ruisleipä – or rye bread – is a staple of the Finnish diet. While there are many varieties, the most popular and widely available is reikäleipä, meaning “bread with a hole.” People used to hang their bread on poles from the rafters, and while it’s dense, flat and very heavy, many Finns will actually have it sent through the post while they’re living abroad. Näkkileipä is the cracker version of rye bread and there are many varieties, including the internationally-sold Finn Crisp cracker. These are eaten at breakfast with butter, cheese and other spreads, with soups at lunch or even as an evening snack.; Rieska – Finnish flatbreads: Rieska is made with dough from a variety of local grains that’s then shaped into a chapati-like bread (or “kovaohranen”). If they’re baked with potato dough, they’re called “lepuska.” The magic of this flatbread lies in its freshness. Prepared on the same day they’re consumed, these savoury delights go great with heaps of butter and scorching black coffee. Tip: Try a different flatbread in each place you visit. Hopefully, you’ll have the chance to taste the local Savo variety, which is made with sour milk.; Korvapuusti – a pastry enjoyed with a cup of coffee: Korvapuusti translates to “slapped ears” in English, but these pastries are essentially cinnamon buns. And while Finland doesn’t hold a patent on cinnamon buns, it probably should. Usually enjoyed with a cup of coffee (Finns consume more coffee and, perhaps, more cinnamon buns than any other European nation), it can be difficult to stop at just one. Or two.; Bilberry pie – adored by all: In July and August, bilberries paint the Finnish forest. You’ll want to pick and freeze them for winter (like the Finns do), but these berries are best enjoyed in the summer months, whether on their own or in homemade pies. Although most Finnish berries can be made into pie filling, bilberry pie, or mustikkapiirakka, served with fresh milk, is one that’s adored by all.; Chocolate from Finland: Chocolate isn’t exactly a Finnish invention, but it’s prepared and sold all over the country. One place to enjoy it is at Brunberg, a famous chocolate shop in Porvoo. In operation since 1871, this family-owned and operated business offers a wide variety of tasty treats. Favourites include Truffles and Kisses, which have become hits among those who love high-quality chocolate. Fazer, another family-operated chocolate brand, can be found in any Finnish café, supermarket or kiosk – their products are even sold as souvenirs in the airport’s duty-free shop. The company produces the country’s most iconic chocolate, Fazerin Sininen, which is loosely translated and commonly known as “the Blue.” This chocolate, along with Fazer’s licorices and other sweets, are frequently shipped abroad, and their company-operated cafés are scattered throughout Finland. They serve Karelian pies and salmon soup, too!
        '''
    ),
    Document(
        page_content='''
        Guide to driving in Finland Here are some tips for driving in Finland – rain, snow or shine: While Finland’s roads are typically in good condition, Nordic weather occasionally throws drivers a curveball. Whether you encounter reindeer on the road in Lapland or ice…well, pretty much anywhere, here’s some practical information for a smoother, safer road trip.; Renting a car: There are car hire companies in all major towns and cities, as well as at airports. It’s worth booking in advance, and you must have a credit card on you when you book. If you have an EU or Swiss drivers’ licence, you can hit the road in Finland straight away.; Avoiding traffic: Traffic jams aren’t common in Finland. Although you might hear some complaints about traffic from Finns, it’s rare to hear them from those who live in big cities elsewhere. To a Finn, a few-minute delay is the equivalent of heavy traffic.; Speed limits: Generally, the speed limit in Finland is 50 km/h in built-up areas and 80 km/h outside of them. Both limits are typically enforced if there is no other speed limit posted. On major highways in summer, you can drive 100 km/h. On motorways, the summertime limit is 120 km/h. In winter, however, the speed limit on highways is reduced to 80 km/h, and on motorways, it’s 100 km/h.; Turn your lights on!: If an oncoming vehicle flashes its high beams at you, it usually means you don’t have your headlights on. It’s a law in Finland that drivers must have their lights on – no matter the season and even in the midnight sun. But being flashed could also mean there’s an accident ahead or an animal is on the roadway, so be sure to stay alert.; Paying (or not paying) tolls: There are zero toll roads or bridges in Finland. Yes, zero. That’s because constructing and maintaining roads and highways is mostly funded by Finnish taxes. So go on, enjoy the drive – it’s on us!; Traffic enforcement cameras: Finnish police use automatic traffic surveillance equipment on heavily trafficked roads and in urban areas. You’ll know you’re driving on one of these roads if you see a yellow road sign depicting a camera.; Beware of the animals: Drivers in Finland should stay alert for signs warning of elk crossing the roadways. If you see one, slow down, and at night, use high beams whenever possible. While many of the country’s major highways are lined with high elk fences, accidents do happen – and they can be fatal for drivers, passengers and animals. Also, if you’re driving in Lapland, a reindeer (or twelve!) on the road is a very common sight. Please make sure you drive carefully and adjust your speed to account for limited visibility and potentially dangerous driving conditions.; Essentials of driving in winter: The downside of the delightful winter weather in Finland is that, at times, roads can be very icy. Because of this, all vehicles must be equipped with winter tyres between December 1st and March 1st. You might even want to have studded tires if you’re driving in the northern part of the country. Also, be sure to set aside adequate time for driving in winter. When the roads are icy or there’s a heavy snowfall, the only way to go about it safely is to drive slowly. Note that roads are generally not salted. Instead, they’re cleared by snow ploughs. When driving an electric car, be aware that cold weather and cabin heating cut your driving power. And whether your car is powered by electricity or gasoline, it’s always a good idea to wear or bring warm clothes with you when driving in Finland – just in case you need to make an unexpected stop.
        '''
    ),
]

**Control Variables for Simple Experiment Data**

For simplicity sake, we
- set the number of documents retrieved to be 10
- set the number of documents to be returned after re-ranking to be 3

Fix the embeddings model and vector store index for FAISS based on our prior research
- embeddings model: all-mpnet-base-v2
- index for FAISS: IndexFlatL2

Fix the document chunk/splitting method based on our prior research (Placed after reading in the data)
  - RecursiveCharacterTextSplitter
    - chunk_size=1000
    - chunk_overlap=200

Fix the Cross-Encoder/Re-ranker model based on our prior research: BAAI/bge-reranker-large


In [7]:
num_docs_retrieved = 10
top_k = 3
bi_encoder_embeddings_model_name = "sentence-transformers/all-mpnet-base-v2" # https://www.sbert.net/docs/sentence_transformer/pretrained_models.html

# Initialise the FAISS retriever
embeddings_model = HuggingFaceEmbeddings(model_name=bi_encoder_embeddings_model_name)
vector_store_index = faiss.IndexFlatL2(len(embeddings_model.embed_query("hello world")))
index = faiss.IndexFlatL2(len(embeddings_model.embed_query("hello world")))
faiss_vector_store = FAISS(
    embedding_function=embeddings_model,
    index=vector_store_index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
docs = text_splitter.split_documents(docs)

cross_encoder_embedings_model_name = "BAAI/bge-reranker-large" # https://huggingface.co/BAAI/bge-reranker-large?library=transformers, https://www.llamaindex.ai/blog/boosting-rag-picking-the-best-embedding-reranker-models-42d079022e83

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
faiss_vector_store.add_documents(docs)
retriever = faiss_vector_store.as_retriever(search_type="similarity", search_kwargs={"k": num_docs_retrieved})

**Test Query 1 for Reranking**

In [9]:
test_query_1 = "Please provide me with the best accomodation in Finland"

**Before re-ranking**

In [10]:
query = test_query_1
retriever.invoke(query)

[Document(metadata={'start_index': 9}, page_content="10 new unique places to stay in Finland From modern wooden cabins and brand-new glass huts to resorts with a touch of luxury: Finland offers a plethora of unique accommodation options that showcase the country's breathtaking natural beauty both in the materials and the locations. In recent years, there has been upsurge in unique new hotels, resorts, and cabins opening up around the country. We've curated a list of the coolest and most recently opened ones where you can immerse yourself in Finnish nature and find your inner happiness. Article published in June 2023.; 1. Hilltop Forest – Inkoo, Helsinki region: Hilltop Forest, a small resort in Inkoo, offers charming triangular wooden huts with large windows facing a lush forest. Curl up in the fresh linen bedding and admire the view over the majestic pine trees and birches. The sole mission of the day is to stroll through the forest to the nearby spa. The resort's restaurant serves br

Before re-ranking, we can see that second highest ranked document is about driving!

**After re-ranking**

In [11]:
cross_encoder_model = HuggingFaceCrossEncoder(model_name=cross_encoder_embedings_model_name)
compressor = CrossEncoderReranker(model=cross_encoder_model, top_n=top_k)
reranked_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)
reranked_retriever.invoke("Please provide me with the best accomodation in Finland")

config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

[Document(metadata={'start_index': 1602}, page_content='vibrant art studios, quaint shops, and idyllic gardens. Guests can explore the local craftsmanship, indulge in delicious Finnish cuisine, or simply immerse themselves in the serene ambiance. Whether seeking a peaceful retreat or a cultural adventure, the Torby Hotel and Fiskars Village promise an unforgettable experience.; 3. Hotel Vihannonkulma – Loimaa, Coast and Archipelago: The Hotel Vihannonkulma building, a private home of the car dealer Olli Vihanto, was closed for almost 30 years until the apartment was rediscovered and revived into a boutique hotel opened in 2022. Spend the night in a classic room with sleek Alvar Aalto inspired lines, or in a modern 70s-style room with eccentric details and teak furniture. Enjoy the small-town atmosphere of Loimaa and stroll around in cafés, shops, and historical buildings. Loimaa, located 50 minutes northeast of Turku, serves as a gateway to the surrounding forests and nature.; 4. Kanav

After re-ranking, we can see that the top 3 documents are now all relevant to our query

<br/>
<br/>
<br/>

## **Full experiment to evaluate the improvements of re-ranking**

In this experiment, we evaluate the improvements made by re-ranking using the full data and test queries

**Control Variables**

- Set the number of documents retrieved to be 10
- Set the number of documents to be returned after re-ranking to be 10 for comparison purposes

- Fix the embeddings model and vector store index for FAISS based on our prior research
  - embeddings model: all-mpnet-base-v2
  - index for FAISS: IndexFlatL2

- Fix the document chunk/splitting method based on our prior research (Placed after reading in the data)
  - RecursiveCharacterTextSplitter
    - chunk_size=1000
    - chunk_overlap=200

- Fix the Cross-Encoder/Re-ranker model based on our prior research: BAAI/bge-reranker-large

In [12]:
num_docs_retrieved = 10
top_k = 10
bi_encoder_embeddings_model_name = "sentence-transformers/all-mpnet-base-v2" # https://www.sbert.net/docs/sentence_transformer/pretrained_models.html

# Initialise the FAISS retriever
embeddings_model = HuggingFaceEmbeddings(model_name=bi_encoder_embeddings_model_name)
vector_store_index = faiss.IndexFlatL2(len(embeddings_model.embed_query("hello world")))
index = faiss.IndexFlatL2(len(embeddings_model.embed_query("hello world")))
faiss_vector_store = FAISS(
    embedding_function=embeddings_model,
    index=vector_store_index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

cross_encoder_embedings_model_name = "BAAI/bge-reranker-large" # https://huggingface.co/BAAI/bge-reranker-large?library=transformers, https://www.llamaindex.ai/blog/boosting-rag-picking-the-best-embedding-reranker-models-42d079022e83

**Experiment Test Queries**

In [13]:
test_queries = ['what is the best food to eat in Finland?', 'what is the best food to eat in Iceland?']

**Setup folder structure and filepaths**

In [14]:
article_names = ['finland_articles_updated.csv', 'iceland_articles_updated.csv']
article_fps = [os.path.join('.', 'data', article_name) for article_name in article_names]

In [15]:
output_folder_binary_normal = os.path.join(os.getcwd(), 'experiment_2_output', 'normal', 'binary_relevance')
output_folder_score_normal = os.path.join(os.getcwd(), 'experiment_2_output', 'normal', 'score_relevance')
output_folder_binary_with_reranking = os.path.join(os.getcwd(), 'experiment_2_output', 'reranked', 'binary_relevance')
output_folder_score_with_reranking = os.path.join(os.getcwd(), 'experiment_2_output', 'reranked', 'score_relevance')
output_folders = [output_folder_binary_normal, output_folder_score_normal, output_folder_binary_with_reranking, output_folder_score_with_reranking]

for output_folder in output_folders:
  os.makedirs(output_folder, exist_ok=True)

**Run the experiment for all data**

In [16]:
docs = []
for article_fp in article_fps:
  df = pd.read_csv(article_fp)
  for _, row in df.iterrows():
    text = row['Title'] + " " + row['Content']

    doc = Document(
        page_content=text,
        metadata={'country': row['Country'], 'source': row['Source'], 'link': row['Article Links']}
    )

    docs.append(doc)

In [17]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
chunked_docs = text_splitter.split_documents(docs)

In [18]:
faiss_vector_store.add_documents(docs)
retriever = faiss_vector_store.as_retriever(search_type="similarity", search_kwargs={"k": num_docs_retrieved})

In [19]:
# Run the experiment without re-ranking first

f_name_binary_normal = os.path.join(output_folder_binary_normal, 'normal_retriever_binary_relevance.csv')
f_name_score_normal = os.path.join(output_folder_score_normal, 'normal_retriever_score_relevance.csv')
results = []

for q in test_queries:
    retrieved_docs = [d.page_content for d in retriever.invoke(q)]

    for idx,retrieved_doc in enumerate(retrieved_docs):
        results.append({
            'idx': idx,
            'query': q,
            'retrieved_doc': retrieved_doc,
            'relevant': ''
        })

results_df = pd.DataFrame(results)
results_df.to_csv(f_name_binary_normal, index=False)
results_df.to_csv(f_name_score_normal, index=False)

In [20]:
cross_encoder_model = HuggingFaceCrossEncoder(model_name=cross_encoder_embedings_model_name)
compressor = CrossEncoderReranker(model=cross_encoder_model, top_n=top_k)
reranked_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [21]:
# Run the experiment with re-ranking

f_name_binary_with_reranking = os.path.join(output_folder_binary_with_reranking, 'reranked_retriever_binary_relevance.csv')
f_name_score_with_reranking = os.path.join(output_folder_score_with_reranking, 'reranked_retriever_score_relevance.csv')
results = []

for q in test_queries:
    retrieved_docs = [d.page_content for d in reranked_retriever.invoke(q)]

    for idx,retrieved_doc in enumerate(retrieved_docs):
        results.append({
            'idx': idx,
            'query': q,
            'retrieved_doc': retrieved_doc,
            'relevant': ''
        })

results_df = pd.DataFrame(results)
results_df.to_csv(f_name_binary_with_reranking, index=False)
results_df.to_csv(f_name_score_with_reranking, index=False)

Your folder structure should now look as such:

```
data
  - iceland_articles_updated.csv
  - finland_articles_updated.csv

experiment_2_output
  - normal
      - binary_relevance
        - normal_retriever_binary_relevance.csv
      - score_relevance
        - normal_retriever_binary_relevance.csv
  - reranked
      - binary_relevance
        - reranked_retriever_binary_relevance.csv
      - score_relevance
        - reranked_retriever_score_relevance.csv
```

**Save the experiment output**

In [22]:
%cd /content/

/content


In [23]:
!zip -r experiment_2_output.zip experiment_2_output

  adding: experiment_2_output/ (stored 0%)
  adding: experiment_2_output/reranked/ (stored 0%)
  adding: experiment_2_output/reranked/binary_relevance/ (stored 0%)
  adding: experiment_2_output/reranked/binary_relevance/reranked_retriever_binary_relevance.csv (deflated 62%)
  adding: experiment_2_output/reranked/score_relevance/ (stored 0%)
  adding: experiment_2_output/reranked/score_relevance/reranked_retriever_score_relevance.csv (deflated 62%)
  adding: experiment_2_output/normal/ (stored 0%)
  adding: experiment_2_output/normal/binary_relevance/ (stored 0%)
  adding: experiment_2_output/normal/binary_relevance/normal_retriever_binary_relevance.csv (deflated 62%)
  adding: experiment_2_output/normal/score_relevance/ (stored 0%)
  adding: experiment_2_output/normal/score_relevance/normal_retriever_score_relevance.csv (deflated 62%)


In [24]:
files.download("experiment_2_output.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>